# Текстовое ранжирование с помощью библиотеки Whoosh

В этом примере мы рассмотрим как с помощью библиотеки **whoosh** можно:
- проиндексировать коллекцию текстовых документов
- организовать по ней поиск, в котором будет использоваться ранжирование по формуле BM25
- посчитать метрики качества такого поиска

Будем использовать датасет **VK MARCO**.

Этот датасет был получен путем обкачки выдачи веб-поиска, который разрабатывается компанией VK, с последующей асессорской оценкой этой выдачи.

Другими словами, датасет содержит:
- поисковые запросы
- тексты веб-страниц (уже с очищенной HTML-разметкой)
- оценки релевантности для пар запрос-документ. Оценки выставлялись по 4-х балльной шкале, от 0 до 3 (включительно). Также, обратите внимание, что оценивался только топ-K поисковой выдачи (где K может варьироваться, как правило это несколько десятков документов), т.е. все оцененные документы априори являются довольно релевантными (иначе бы их не было в топе).

Датасет хранится в том же формате, что и популярный поисковый датасет <a href="https://microsoft.github.io/msmarco/">MS MARCO</a> от компании Microsoft (и названии позаимствовано отсюда же).

Т.к. датасет VK MARCO довольно большой, то в этом примере мы будем использовать только маленький сэмпл из ~9 тыс. документов, а с полным датасетом вам предстоит столкнуться в ближайших ДЗ (про текстовую релевантность, нейросетевый модели и в финальном проекте).

Весь код этого семинара, в т.ч. код загрузки данных, можно смело переиспользовать в своих решениях ДЗ.

Давайте начнем с того, что скачаем наш сэмпл и положим в папку data в корне проекта (ссылка на данные будет предоставлена отдельно чтобы не "светить" датасет на гитхабе):

In [56]:
!ls ../../data/text-ranking-seminar-vk-ir-fall-2024

vkmarco-docdev-qrels.tsv.gz    vkmarco-doctrain-qrels.tsv.gz
vkmarco-docdev-queries.tsv.gz  vkmarco-doctrain-queries.tsv.gz
vkmarco-docs.tsv


Рассмотрим подробнее формат датасета, нас тут в первую очередь интересуют 3 файла:
- _vkmarco-docs.tsv_              - в нем лежат 9025 документов, по которым будем искать, в формате _DOC_ID_\t_URL_\t_TITLE_\t_BODY_
- _vkmarco-docdev-queries.tsv.gz_ - тут лежат 100 запросов, по которым мы хотим искать, в формате _QUERY_ID_\t_QUERY_
- _vkmarco-docdev-qrels.tsv.gz_   - тут хранятся оценки для пар запрос-документ, в формате _QUERY_ID_ 0 _DOC_ID_ _LABEL_

Тут:
- \t         - это просто разделитель (табуляция)
- _DOC_ID_   - это уникальный идентификатор документа, он имеет вид строки типа "D2749594"
- _QUERY_ID_ - это уникальный идентификатор запроса, это просто число, например 42568
- _LABEL_    - это асессорская оценка релевантности, у нас она может принимать значения от 0 до 3 (включительно).

Документы состоят из 2х зон: _TITLE_ и _BODY_

Важный момент: все пары запрос-документ, которых не было в файлике _vkmarco-docdev-qrels.tsv.gz_, мы в дальнейшем считаем нерелевантными! Но это, в общем случае, неверно, т.к. асессоры оценивали не все возможные пары запрос-документ, а только те документы, которые были в топе по данному запросу. Это типичная проблема, с которой сталкиваются разработчики поисковых систем, и нам придется с этим жить. Но на практике большинство таких документов действительно не являются релевантными.

Теперь попробуем загрузить наш датасет.

## Загружаем датасет VK MARCO

Импортируем все что понадобится для дальнейшей работы:

In [57]:
import math
import pathlib
import shutil

import pandas as pd

from whoosh import analysis
from whoosh import fields
from whoosh import index
from whoosh.lang import porter
from whoosh.lang.snowball import english, russian
from whoosh import qparser

### Загружаем запросы

In [58]:
# Путь до корня проекта
project_root_dir = pathlib.Path("../..")

# Путь до датасета
data_dir = project_root_dir.joinpath("data/text-ranking-seminar-vk-ir-fall-2024")

# Файл с запросами
queries_file = data_dir.joinpath(f"vkmarco-docdev-queries.tsv.gz")

# Загружаем запросы во фрейм
queries_df = pd.read_csv(queries_file, sep='\t', header=None)
queries_df.columns = ['query_id', 'query_text']
print(queries_df.head(5))

   query_id                                         query_text
0     12509                                      рев компонент
1      9924              ооо мебельная компания в калининграде
2      8719  можно ли снижать бал на математике за граматич...
3     37469                 реставрация домов тухачевского спб
4      5521         как в смете взять работу по контробрешетки


Обратите внимание, что это реальные запросы реальных пользователей, которые предоставлены as is, т.е. в них могут встречаться:
- adult content
- опечатки (обратите внимание на запрос "можно ли снижать бал на математике за граматические рщиьки")
- и много другого, странного, необъяснимого и подчас шокирующего -- будьте к этому (морально) готовы!

### Загружаем оценки

Теперь загрузим т.н. qrels -- асессорские оценки релевантности для пар запрос-документ:

In [59]:
# Файл с оценками
qrels_file = data_dir.joinpath(f"vkmarco-docdev-qrels.tsv.gz")

# Загружаем оценки во фрейм
qrels_df = pd.read_csv(qrels_file, sep=' ', header=None)
qrels_df.columns = ['query_id', 'unused', 'docid', 'label']
print(qrels_df.head(5))

   query_id  unused       docid  label
0       283       0  D000007265      1
1       283       0  D000007266      2
2       283       0  D000007267      0
3       283       0  D000007268      2
4       283       0  D000007269      3


### Конвертируем запросы и оценки в более удобный для дальнейшей работы формат

In [60]:
# Представляем запросы в виде словаря: Query ID -> Text
query_id2text = {query_id: query_text for query_id, query_text in zip(queries_df['query_id'], queries_df['query_text'])}
print(query_id2text)

{12509: 'рев компонент', 9924: 'ооо мебельная компания в калининграде', 8719: 'можно ли снижать бал на математике за граматические рщиьки', 37469: 'реставрация домов тухачевского спб', 5521: 'как в смете взять работу по контробрешетки', 34024: 'как изменить размер изображения с помощью adobe photoshop', 16023: 'училка на уроке анатомии разделась', 28016: 'карантин в хабаровском крае', 6382: 'какой день недели был 20 августа 1987 года', 39674: 'простые схемы ажура спицами', 20929: 'фирма сантекс город пенза', 12742: 'рифма к слову причастье', 36197: 'л 1222 8400015 маска капота мтз 1523 беларусьиз чего сделана', 283: '9729101772', 3202: 'всд и емс тренировки', 29488: 'яке виділення богомола', 22559: 'собакевич продал души чичиковву', 37606: 'тест 8 тема решение уравнений вариант 1', 15072: 'сырники в духовке как пропечь с двух сторон', 19938: 'поселок авангард самара', 36880: 'анатолий курносов художник пудож', 30784: 'как принимать ланцид кит', 9785: 'один в прошлом смотреть', 17415: '

In [61]:
# Представляем оценки в виде словаря: Query ID -> Doc ID -> Label (relevance)
qrels = {}
for i in range(0, len(qrels_df)):
    qrels_row = qrels_df.iloc[i]
    query_id = qrels_row['query_id']
    docid = qrels_row['docid']
    label = qrels_row['label']
    if label < 0 or label > 3:
        raise Exception(f"invalid label in qrels: doc_id = {doc_id}")

    docid2label = qrels.get(query_id)
    if docid2label is None:
        docid2label = {}
        qrels[query_id] = docid2label
    docid2label[docid] = label
print(qrels)

{283: {'D000007265': 1, 'D000007266': 2, 'D000007267': 0, 'D000007268': 2, 'D000007269': 3, 'D000007270': 3, 'D000007271': 3, 'D000007272': 2, 'D000007273': 1, 'D000007274': 3}, 837: {'D000019133': 1, 'D000019134': 1, 'D000019135': 1, 'D000019136': 1, 'D000019137': 2, 'D000019138': 1, 'D000019139': 1, 'D000019140': 1, 'D000019141': 1, 'D000019142': 1, 'D000019143': 1, 'D000019144': 1, 'D000019145': 1, 'D000019146': 1, 'D000019147': 1, 'D000019148': 1, 'D000019149': 1, 'D000019150': 1, 'D000019151': 1, 'D000019152': 1, 'D000019153': 1, 'D000019154': 1, 'D000019155': 1, 'D000019156': 1, 'D000019157': 1, 'D000019158': 1, 'D000019159': 1, 'D000019160': 1, 'D000019161': 1, 'D000019162': 1, 'D000019163': 1}, 872: {'D000019917': 1, 'D000019918': 1, 'D000019919': 2, 'D000019920': 1, 'D000019921': 1, 'D000019922': 1, 'D000019923': 2, 'D000019924': 2, 'D000019925': 1, 'D000019926': 0, 'D000019927': 1, 'D000019928': 1, 'D000019929': 1, 'D000019930': 1, 'D000019914': 1, 'D000019931': 1}, 1881: {'D

### Загружаем документы

Напишем функцию для загрузки документов:

In [62]:
# Функция которая читает все документы в один большой список
def read_docs(docs_file):
    docs = []
    with open(docs_file, 'rt') as f:
        for line in f:
            # Парсим следующую строку
            parts = line.rstrip('\n').split('\t')
            docid, url, title, body = parts

            # Валидируем
            if not docid:
                raise RuntimeError(f"invalid doc id: num_lines = {self.num_lines}")
            if not url:
                raise RuntimeError(f"invalid url: num_lines = {self.num_lines}")
           
            # Пакуем данные документа в словарь
            doc = {'url': url, 'title': title, 'body': body, 'docid': docid}
            docs.append(doc)
    return docs

И теперь загрузим наши документы:

In [63]:
# Файл с документами
docs_file = data_dir.joinpath("vkmarco-docs.tsv")

# Загружаем все документы
docs = read_docs(docs_file)
print(f"Loaded {len(docs)} docs")

Loaded 9025 docs


## Индексируем документы

### Готовимся к индексации

Перед индексацией нам надо подготовить:
- Объект-анализатор, который будет определять как именно мы будем нормализовывать текст, в частности хотим ли мы использовать стемминг
- Схему индекса, которая определяет из каких текстовых зон будет состоять наш документ

In [64]:
# Создаем Analyzer с поддержкой стемминга для английского языка
#stemmer = english.EnglishStemmer()
#stemfn = stemmer.stem
#analyzer = analysis.StemmingAnalyzer(stemfn=stemfn)

# Создаем analyzer (без поддержки стемминга) который будет использоваться для обработки текстов запроса и документа
analyzer = analysis.StandardAnalyzer()

# Создаем схему индекса: будем хранить Doc ID, URL, и тексты TITLE и BODY
schema = fields.Schema(
    docid=fields.ID(stored=True), 
    url=fields.TEXT(stored=True),
    title=fields.TEXT(analyzer=analyzer),
    body=fields.TEXT(analyzer=analyzer)
)

### Создаем и заполняем индекс

In [65]:
# Временная папка для индекса
index_dir = pathlib.Path("/tmp/index")

# Удаляем старый индекс, если такой существует
shutil.rmtree(index_dir, ignore_errors=True)

# Создаем заново папку под индекс
index_dir.mkdir(exist_ok=True, parents=True)

# Создаем индекс согласно объявленной схеме
ix = index.create_in(index_dir, schema)

# Объект-writer который будет использоваться для добавления новых документов в индекс
writer = ix.writer()

# Добавляем все наши документы в индекс
num_docs = 0
for doc in docs:
    num_docs += 1
    writer.add_document(docid=doc['docid'], url=doc['url'], title=doc['title'], body=doc['body'])
    if num_docs % 100 == 0:
        print(f"added {num_docs} docs...", end='\r')
print(f"added all ({num_docs}) docs, committing index...")

# Записываем на диск
writer.commit()
print("index committed")

added all (9025) docs, committing index...
index committed


Посмотрим на структуру индекса:

In [66]:
!ls /tmp/index

_MAIN_1.toc  MAIN_wblu4xvyphkatk5p.seg	MAIN_WRITELOCK


## Ищем запросы в индексе

Подготовим функцию для рассчета метрики DCG:

In [67]:
# Функция для подсчета DCG@K, понадобится нам для расчета метрик
def dcg(y, k=10):
    """Computes DCG@k for a single query.
            
    y is a list of relevance grades sorted by position.
    len(y) could be <= k.
    """     
    r = 0.
    for i, y_i in enumerate(y):
        p = i + 1 # position starts from 1
        r += (2 ** y_i - 1) / math.log(1 + p, 2)
        if p == k:
            break
    return r

Прогоним наши запросы через индекс и рассчитаем метрики:

In [68]:
# Готовим парсер запросов. Будем искать сразу в 2х полях (TITLE и BODY) используя т.н. кворум (булев поиск с "мягким И")
qp = qparser.MultifieldParser(['title', 'body'], schema=ix.schema, group=qparser.OrGroup.factory(0.9))
qp.remove_plugin_class(qparser.WildcardPlugin) # Ускоряет поиск

# Суммарный DCG@10 по всем запросам
dcg10_sum = 0

# Вспомогательная функция, которая превращает None в 0
def none_to_label(label):
    return 0 if label is None else label
    
# Создаем объект searcher с помощью которого будем искать в индексе
with ix.searcher() as searcher:
    # Ищем каждый запрос по очереди
    query_ids = sorted(query_id2text.keys())
    for query_id in query_ids:
        query_text = query_id2text[query_id]
        
        # Парсим запрос
        query = qp.parse(query_text)

        # Собственно сам поиск
        results = searcher.search(query)

        # Достаем из результатов поиска т.н. "хиты" -- найденные документы.
        # Результаты уже отранжированы с помощью формулы BM25!
        num_hits = results.scored_length()
        hits = [results[i] for i in range(num_hits)]
        #print(hits)

        # Найденные Doc ID
        found_docids = [hit['docid'] for hit in hits]

        # Получаем все известные оценки для этого запроса
        docid2label = qrels[query_id]

        # Формируем сортированный по убыванию ранка (BM25) список оценок для всех найденных документов
        labels = [none_to_label(docid2label.get(docid)) for docid in found_docids]

        # Считаем DCG@10 для этого запроса
        dcg10 = dcg(labels, k=10)
        dcg10_sum += dcg10
        print(f"Next query: query_id = {query_id} query_text = '{query_text}' found_docids = {found_docids} labels = {labels} dcg@10 = {dcg10:.3f}")
        print(docid2label)

Next query: query_id = 283 query_text = '9729101772' found_docids = ['D000007274', 'D000007269', 'D000007271', 'D000007266', 'D000007270', 'D000007273', 'D000007268', 'D000007267', 'D000007265', 'D000007272'] labels = [3, 3, 3, 2, 3, 1, 2, 0, 1, 2] dcg@10 = 21.441
{'D000007265': 1, 'D000007266': 2, 'D000007267': 0, 'D000007268': 2, 'D000007269': 3, 'D000007270': 3, 'D000007271': 3, 'D000007272': 2, 'D000007273': 1, 'D000007274': 3}
Next query: query_id = 837 query_text = 'https m ok ru dk st cmd friendallphotos st friendid 244176175253 prevcmd friendmain tkn 6883' found_docids = ['D000019134', 'D000019136', 'D000019150', 'D000019154', 'D000019145', 'D000019144', 'D000019148', 'D000019158', 'D000019138', 'D000019140'] labels = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] dcg@10 = 4.544
{'D000019133': 1, 'D000019134': 1, 'D000019135': 1, 'D000019136': 1, 'D000019137': 2, 'D000019138': 1, 'D000019139': 1, 'D000019140': 1, 'D000019141': 1, 'D000019142': 1, 'D000019143': 1, 'D000019144': 1, 'D000019145':

### Считаем средние метрики

И теперь усредним нашу метрику по запросам:

In [69]:
# Полное число запросов
num_queries = len(query_id2text)

# Среднее DCG@10
dcg10_avg = dcg10_sum / num_queries
print(dcg10_avg)

11.702716619579123


Кажется, что мы получили довольно качественный поиск!